In [ ]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

import datetime
import time
import loader
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
symbol = 'AAPL'
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta( days = 3600 )

In [ ]:
# load from csv
df = loader.load_stocks(symbol, start_date, end_date)
df.head()

In [ ]:
X_train, Y_train, X_test, Y_test = loader.pre_pros_as_sequences(df, 50)
print(X_train.shape)

In [ ]:
model = Sequential()

model.add(LSTM(input_shape = X_train.shape[1:], units = 50, return_sequences = True, name='LSTM_1'))
model.add(Dropout(0.2))

model.add(LSTM(units = 100, return_sequences = False, name='LSTM_2'))
model.add(Dropout(0.2))

model.add(Dense(units = 1))
model.add(Activation('linear'))

start = time.time()
model.compile(loss = 'mse', optimizer = 'rmsprop', metrics=['acc'])
print('compilation time : ', time.time() - start)
print(model.summary())

In [ ]:
model.fit(X_train, Y_train, batch_size = 512, epochs = 1, validation_split = 0.05)

In [ ]:
window_size = 50
prediction_len = 50
data = X_test

prediction_seqs = []
for i in range(int(len(data) /  prediction_len)):
    curr_frame = data[i * prediction_len]
    predicted = []
    for j in range(prediction_len):
        predicted.append(model.predict(curr_frame[np.newaxis,:,:])[0,0])
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

In [ ]:
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)
ax.plot(Y_test, label ='True Data')

for i, data in enumerate(prediction_seqs):
    padding = [None for p in range(i * prediction_len)]
    plt.plot(padding + data, label ='Prediction')
    plt.legend()
plt.show()